In [1]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

import sys 
# sys.path.append('C:\\Users\\analytic6\\Desktop\\Work Space Analitic 6 (Asir)\\')
# sys.path.append('C:\\Users\\User\\Desktop\\work')

from AMLpp.transformers import Word2Vectorization, ImputerIterative, CategoricalEncoder
from AMLpp.conveyor import Conveyor
from AMLpp.architect import Experimenter

from CustomTransform import Transform_newubki_ext

from typing import List
import pandas as pd

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\analytic6\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\analytic6\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
path_trainset = 'test.xlsx'
df = pd.read_excel(path_trainset)

In [4]:
d = df.copy()

In [4]:
print('Количество с дупликатами = {}'.format(len(d)))
d = d.drop_duplicates(subset=['backend_application_id'], keep='last')
print('Количество без дупликатов = {}'.format(len(d)))

Количество с дупликатами = 27623
Количество без дупликатов = 27623


In [5]:
d['date_app'] = pd.to_datetime(d['applied_at'], format='%Y-%m-%d', errors='coerce')
for year in range(2018,2022):
    print("*"*100)
    quantity = len(d[d['date_app'].dt.year == year])
    print(f"Количество заявок {year} года = {quantity}")

    month_quantity = ""
    for month in range(1, 13):
        quantity = len(d[(d['date_app'].dt.month == month) & (d['date_app'].dt.year == year)])
        month_quantity += f"месяц [{month}] = {quantity}, "
    print(month_quantity)

****************************************************************************************************
Количество заявок 2018 года = 6848
месяц [1] = 38, месяц [2] = 16, месяц [3] = 14, месяц [4] = 22, месяц [5] = 20, месяц [6] = 868, месяц [7] = 832, месяц [8] = 456, месяц [9] = 497, месяц [10] = 1072, месяц [11] = 1704, месяц [12] = 1309, 
****************************************************************************************************
Количество заявок 2019 года = 18514
месяц [1] = 1823, месяц [2] = 2368, месяц [3] = 2179, месяц [4] = 1645, месяц [5] = 1466, месяц [6] = 999, месяц [7] = 1085, месяц [8] = 1047, месяц [9] = 1492, месяц [10] = 1483, месяц [11] = 1553, месяц [12] = 1374, 
****************************************************************************************************
Количество заявок 2020 года = 2244
месяц [1] = 1725, месяц [2] = 513, месяц [3] = 6, месяц [4] = 0, месяц [5] = 0, месяц [6] = 0, месяц [7] = 0, месяц [8] = 0, месяц [9] = 0, месяц [10] = 0, месяц [11]

In [17]:
separation_expression = (d['date_app'].dt.year < 2020) & ((d['date_app'].dt.year < 2019 ) | (d['date_app'].dt.month != 12))
x_train = d[separation_expression].copy()
y_train = x_train[['overdue_days', 'status_id']].copy()

x_test = d[~separation_expression].copy()
y_test = x_test[['overdue_days', 'status_id']].copy()

In [12]:
test_name = "из тренировочного сета, в промежутки с 2019-го года, от 12-го месяца"
print(f"Общее количество тренировочных {len(x_train)}")
print('count status 2 = {}'.format(len(x_train[x_train['status_id'] == 2])))
print('count status 5 = {}'.format(len(x_train[x_train['status_id'] == 5])))
print('count status 6 = {}'.format(len(x_train[x_train['status_id'] == 6])))
print(f"Соотношение просроченых к хорошим, {len(x_train[x_train['overdue_days'] > 0])}/{len(x_train[x_train['overdue_days'] == 0])}")

print(f"Общее количество тестовых {len(x_test)}")
print('count status 2 = {}'.format(len(x_test[x_test['status_id'] == 2])))
print('count status 5 = {}'.format(len(x_test[x_test['status_id'] == 5])))
print('count status 6 = {}'.format(len(x_test[x_test['status_id'] == 6])))
print(f"Соотношение просроченых к хорошим, {len(x_test[x_test['overdue_days'] > 0])}/{len(x_test[x_test['overdue_days'] == 0])}")

Общее количество тренировочных 23988
count status 2 = 10848
count status 5 = 8798
count status 6 = 4342
Соотношение просроченых к хорошим, 7954/5186
Общее количество тестовых 3635
count status 2 = 42
count status 5 = 2535
count status 6 = 1058
Соотношение просроченых к хорошим, 2073/1520


In [13]:
categorical_columns = ['organization_type_other', 'position_other', 'email', 'purpose_other', 'ceduc', 'family', 'sstate', 'cgrag', 'income_source_id',
                      'income_frequency_id', 'has_prior_employment', 'empoyees_count_id', 'organization_branch_id', 'organization_type_id', 'position_id',
                      'employment_type_id', 'has_movables', 'has_immovables', 'fact_addr_owner_type_id', 'fact_addr_region_id', 'fact_addr_same', 
                      'addr_region_id', 'education_id', 'children_count_id', 'marital_status_id', 'gender_id', 'addr_owner_type_id', 'country_det',
                      'city_det', 'region_det', 'isp', 'browser', 'system', 'brand'
                      ]

from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import PolynomialFeatures
from tpot.builtins import StackingEstimator
from sklearn.linear_model import RidgeCV
from lightgbm import LGBMRegressor

model = Conveyor(
    Transform_newubki_ext(),
    Word2Vectorization(columns=['purpose_other']),
    CategoricalEncoder(categorical_columns),
    ImputerIterative(),
    PolynomialFeatures(),
    estimator = 
    LGBMRegressor(**{'test_split': 5, 'random_state': 25, 'reg_alpha': 0.14998013314891953,
    'reg_lambda': 3.7370516908015063, 'colsample_bytree': 0.85, 'subsample': 0.7, 
    'learning_rate': 0.03, 'max_depth': 10, 'num_leaves': 281,
    'min_child_samples': 15, 'cat_smooth': 69})
)


In [14]:
model.fit(x_train, y_train)

100%|██████████| 5/5 [03:49<00:00, 45.85s/it, transform=PolynomialFeatures]


[LightGBM] [Warning] Unknown parameter: test_split


In [19]:
model.fit_model(x_train, y_train, x_test, y_test, categorical_columns = categorical_columns)

100%|██████████| 5/5 [03:39<00:00, 43.96s/it, transform=PolynomialFeatures]


[LightGBM] [Warning] Unknown parameter: test_split
start fit tpot model !!!!


In [15]:
experiment = Experimenter("first_model")
# experiment = Experimenter("clUb_new_lgb_optunamax")

In [18]:
description = \
"""
Черновой вариант, с дефолтной моделью lgb regressor с рандомными характеристиками.
"""
testset_name = "Тестировочные данные, из тренировочного сета. Год 2020 и 2019 и месяц 12"
X_test_features = ['backend_application_id', 'overdue_days','status_id']
# experiment.create_experiment(model, description, path_trainset)
experiment.create_experiment(model, description, path_trainset, x_test, y_test, testset_name)#, X_test_features = X_test_features)


************************************************************
testset = Тестировочные данные, из тренировочного сета. Год 2020 и 2019 и месяц 12
function - roc_auc_score = 0.6165322881672266
function - r2_score = 0.02619725758552316
function - accuracy_score = ERROR: Classification metrics can't handle a mix of binary and continuous targets



KeyboardInterrupt: 

In [149]:
experiment.make_experiment(X_test, y_test, "контрольный тест", X_test_features = X_test_features, feature_importances = False)


************************************************************
testset = контрольный тест
function - roc_auc_score = 0.7949683314368754
function - r2_score = -0.42920562604898915
function - accuracy_score = ERROR: Classification metrics can't handle a mix of binary and continuous targets

